# Notebook exploratoire des données de déclaration d'intérêt des élus

[Lien de la source de données](https://www.hatvp.fr/livraison/merge/declarations.xml)

## Introduction sur l'intérêt d'étudier les déclarations d'intérêt des élus

Les déclarations d'intérêt des élus contiennent des informations détaillées sur les intérêts financiers, professionnels et personnels des élus. Elles visent à garantir la transparence et à prévenir les conflits d'intérêts.  
Ces déclarations incluent généralement des informations sur les revenus, les participations financières, les activités professionnelles, les mandats et les fonctions exercées, ainsi que les éventuels liens familiaux pouvant influencer les décisions publiques.

L'objectif de ce notebook est donc d'explorer ces données de déclaration afin d'en dégager des informations pertinentes et éventuellement déterminer un processus pour recueillir automatiquement ces données.

## Importation et aperçu des données

In [4]:
# Bibliothèques
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup



In [9]:
# 2 fichiers sources pour les déclarations
# Liste .csv = en têtes des déclarations avec élus
# Déclaration .xml = contenus des déclarations 

# data = pd.read_parquet("Jeux de données/declarations.parquet")

liste = pd.read_csv('/home/jbn/.jupyter/liste.csv', sep = ';')

In [ ]:
# import du élus.parquet de CGoudet issu du RNE
elus = pd.read_parquet('/home/jbn/.jupyter/elus.parquet')
elus

In [ ]:
declarations = pd.read_parquet('/home/jbn/.jupyter/declarations.parquet')

In [ ]:
# URL du fichier XML (je ne peux pas le télécharger)
url = 'https://www.hatvp.fr/livraison/merge/declarations.xml'

# Requêtage du fichier
response = requests.get(url)
xml_content = response.content

# Parsing du fichier XML
root = ET.fromstring(xml_content)


In [ ]:
# Représentation de l'arbre XML

xml_str = ET.tostring(root, encoding='utf-8').decode('utf-8')


In [ ]:
# Ecriture du fichier XML et enregistrement local
tree = ET.ElementTree(root)
tree.write("file.xml")

In [ ]:
file = 'file.xml'

In [ ]:
# Création de l'objet BeautifulSoup
with open(file, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f.read(), "lxml")  # Utilisation de "lxml"
    print("Soup created")

In [ ]:
# identification des balises déclarations
declarations = soup.find_all("declaration")
len(declarations)

In [ ]:
# Initialisation d'une liste pour stocker les lignes de données de la rubrique général
data_general = []

# Boucle sur les sous-éléments imbriqués (ex: activprofcinqdernieredto, mandatelectifdto, etc.)
for declaration in soup.find_all("declaration") :
    
    DateDepot = declaration.find("datedepot").get_text(strip=True) if declaration.find("datedepot") else None
    Uuid = declaration.find("uuid").get_text(strip=True) if declaration.find("uuid") else None
    Origine = declaration.find("origine").get_text(strip=True) if declaration.find("origine") else None
    Complete = declaration.find("complete").get_text(strip=True) if declaration.find("complete") else None

    for section in declaration.find_all("general") :

        Civilite = section.find("civilite").get_text(strip=True) if section.find("civilite").get_text(strip=True) else None
        Nom = section.find("nom").get_text(strip=True) if section.find("nom").get_text(strip=True) else None
        Prénom = section.find("prenom").get_text(strip=True) if section.find("prenom").get_text(strip=True) else None
        Date_Naissance = section.find("datenaissance").get_text(strip=True) if section.find("datenaissance").get_text(strip=True) else None

        typedeclaration = section.find("typedeclaration")
        Type_Déclaration = typedeclaration.find("id").get_text(strip=True) if typedeclaration and typedeclaration.find("id").get_text(strip=True) else None
        Type_Nom_Déclaration = typedeclaration.find("label").get_text(strip=True) if typedeclaration and typedeclaration.find("label").get_text(strip=True) else None
        Modif_Déclaration = section.find("declarationmodificative").get_text(strip=True) if section.find("declarationmodificative") else None
    
        mandat = section.find("mandat")
        Nom_Mandat = mandat.find("label").get_text(strip=True) if mandat and mandat.find("label").get_text(strip=True) else None
        
        qualitemandat = section.find("qualitemandat")
        Type_Mandat = qualitemandat.find("typemandat").get_text(strip=True) if qualitemandat and qualitemandat.find("typemandat").get_text(strip=True) else None
        Catégorie_Code_Mandat = qualitemandat.find("codcategoriemandat").get_text(strip=True) if qualitemandat and qualitemandat.find("codcategoriemandat").get_text(strip=True) else None
        Catégorie_Nom_Mandat = qualitemandat.find("nomcategoriemandat").get_text(strip=True) if qualitemandat and qualitemandat.find("nomcategoriemandat").get_text(strip=True) else None
        Type_Nom_Mandat = qualitemandat.find("labeltypemandat").get_text(strip=True) if qualitemandat and qualitemandat.find("labeltypemandat").get_text(strip=True) else None
        Organe_Intitulé_Mandat = qualitemandat.find("labelorgane").get_text(strip=True) if qualitemandat and qualitemandat.find("labelorgane").get_text(strip=True) else None
        Organe_CodeListe_Mandat = qualitemandat.find("codelisteorgane").get_text(strip=True) if qualitemandat and qualitemandat.find("codelisteorgane").get_text(strip=True) else None

        organe = section.find("organe")
        Organe_Code = organe.find("codelisteorgane").get_text(strip=True) if organe and organe.find("codelisteorgane") else None
        Organe_Nom = organe.find("nomlisteorgane").get_text(strip=True) if organe and organe.find("nomlisteorgane") else None
        Organe_Intitulé = organe.find("labelorgane").get_text(strip=True) if organe and organe.find("labelorgane") else None
        Organe_Déclaration = organe.find("labeldeclaration").get_text(strip=True) if organe and organe.find("labeldeclaration") else None
        
        Qualité_Déclarant = section.find("qualitedeclarant").get_text(strip=True) if section.find("qualitedeclarant").get_text(strip=True) else None
        Qualité_Déclarant_PDF = section.find("qualitedeclarantforpdf").get_text(strip=True) if section.find("qualitedeclarantforpdf").get_text(strip=True) else None
        Date_début_Mandat = section.find("datedebutmandat").get_text(strip=True) if section.find("datedebutmandat").get_text(strip=True) else None
        Date_fin_Mandat = section.find("datefinmandat").get_text(strip=True) if section.find("datefinmandat").get_text(strip=True) else None
        Date_dernière_déclaration = section.find("datederndeclar").get_text(strip=True) if section.find("datederndeclar").get_text(strip=True) else None
        # Régime_Matrimonial = section.find("regimematrimonial").get_text(strip=True) if section.find("regimematrimonial").get_text(strip=True) else None
        # Régime_Matrimonial_Commentaire = section.find("regimematrimonialcomments").get_text(strip=True) if section.find("regimematrimonialcomments").get_text(strip=True) else None
        # Nom_Société = section.find("nomsociete").get_text(strip=True) if section.find("nomsociete").get_text(strip=True) else None
        # Nom_Société_mère = section.find("nomsocietemere").get_text(strip=True) if section.find("nomsocietemere").get_text(strip=True) else None
        # Chiffre_affaires = section.find("chiffreaffaire").get_text(strip=True) if section.find("chiffreaffaire").get_text(strip=True) else None
        # Nombre_Logements = section.find("nblogements").get_text(strip=True) if section.find("nblogements").get_text(strip=True) else None
    
        # Stocker la ligne à chaque élu de la déclaration
        row = {
            "Uuid": Uuid,
            "DateDepot": DateDepot,
            "Origine": Origine,
            "Complete": Complete,
            "Civilite": Civilite,
            "Nom": Nom,
            "Prénom": Prénom,
            "Date_Naissance": Date_Naissance,
            "Type_Déclaration": Type_Déclaration,
            "Type_Nom_Déclaration": Type_Nom_Déclaration,
            "Modif_Déclaration": Modif_Déclaration,
            "Nom_Mandat": Nom_Mandat,
            "Type_Mandat": Type_Mandat,
            "Catégorie_Code_Mandat": Catégorie_Code_Mandat,
            "Catégorie_Nom_Mandat": Catégorie_Nom_Mandat,
            "Type_Nom_Mandat": Type_Nom_Mandat,
            "Organe_Intitulé_Mandat": Organe_Intitulé_Mandat,
            "Organe_CodeListe_Mandat": Organe_CodeListe_Mandat,
            "Organe_Code": Organe_Code,
            "Organe_Nom": Organe_Nom,
            "Organe_Intitulé": Organe_Intitulé,
            "Organe_Déclaration": Organe_Déclaration,
            "Qualité_Déclarant": Qualité_Déclarant,
            "Qualité_Déclarant_PDF": Qualité_Déclarant_PDF,
            "Date_début_Mandat": Date_début_Mandat,
            "Date_fin_Mandat": Date_fin_Mandat,
            "Date_dernière_déclaration": Date_dernière_déclaration,
            # "Régime_Matrimonial": Régime_Matrimonial,
            # "Régime_Matrimonial_Commentaire": Régime_Matrimonial_Commentaire,
            # "Nom_Société": Nom_Société,
            # "Nom_Société_mère": Nom_Société_mère,
            # "Chiffre_affaires": Chiffre_affaires,
            # "Nombre_Logements": Nombre_Logements
        }
        
        data_general.append(row)
    

# Création du DataFrame
dfg = pd.DataFrame(data_general)



In [ ]:
dfg

In [ ]:
nb_doublons = dfg.duplicated(subset=["Uuid"]).sum()
print(f"Nombre de doublons sur la colonne 'Uuid' : {nb_doublons}")

In [ ]:
doublons = dfg[dfg.duplicated(subset=["Uuid"], keep=False)]
print(doublons)

In [ ]:
# Uuid est la clé primaire, on enlève les uuid doublons
dfg = dfg.drop_duplicates(subset=["Uuid"], keep='first')

In [ ]:
# On trie sur les déclarations d'intérêts (DI) et d'intérêts annexes (DIA) 
dfg = dfg[dfg['Type_Déclaration'].isin(['DI','DIA'])]

In [ ]:
dfg.info()

In [ ]:
dfg.to_csv("dfg", sep='%', encoding='utf-8', index=False)

In [ ]:
# Initialisation d'une liste pour stocker les lignes de données de tous les items
data_items = []

for declaration in soup.find_all("declaration"):
    Uuid = declaration.find("uuid").get_text(strip=True) if declaration.find("uuid") else None

    for section in declaration.find_all([
        # pas de participationDto, pas de collaborateuractifDto, car formats différents
        "activconsultantdto", "activprofcinqdernieredto", "activprofconjointdto", 
        "fonctionbenevoledto", "mandatelectifdto", "participationdirigeantdto", "observationinteretdto"]):

        # Vérifier si l'élément "neant" est présent et s'il est True: On saute cet item et passe au suivant
        neant_tag = section.find("neant")
        if neant_tag and neant_tag.get_text(strip=True).lower() == "true":
            continue  
        
        for item in section.find_all("items"):
            for sub_item in item.find_all("items"):
                ID = sub_item.find("id").get_text(strip=True) if sub_item.find("id") else None
                Label = sub_item.find("label").get_text(strip=True) if sub_item.find("label") else None
                Commentaire = sub_item.find("commentaire").get_text(strip=True) if sub_item.find("commentaire") else None
                Conservee = sub_item.find("conservee").get_text(strip=True) if sub_item.find("conservee") else None
                # dans chaque item même définition mais pas la même balise, regarde les balises identiques dans chaque item
                Description = sub_item.find(["description", "activiteprof", "descriptionactivite", 
                                            "descriptionmandat", "activite", "contenu"]).get_text(strip=True) \
                            if sub_item.find(["description", "activiteprof", "descriptionactivite", 
                                                "descriptionmandat", "activite", "contenu"]) else None
                # dans chaque item même définition mais pas la même balise, regarde les balises identiques dans chaque item
                Employeur = sub_item.find(["nomemployeur", "employeur", "employeurconjoint", 
                                        "nomstructure", "nomsociete"]).get_text(strip=True) \
                            if sub_item.find(["nomemployeur", "employeur", "employeurconjoint", 
                                            "nomstructure", "nomsociete"]) else None
                Date_Début = sub_item.find("datedebut").get_text(strip=True) if sub_item.find("datedebut") else None
                Date_Fin = sub_item.find("datefin").get_text(strip=True) if sub_item.find("datefin") else None
                Rémunération_Type = sub_item.find("brutnet").get_text(strip=True) if sub_item.find("brutnet") else None

                # Boucle sur chaque "montant" pour créer une ligne par montant/année
                for montant in sub_item.find_all("montant"):
                    for sub_montant in montant.find_all("montant"):
                        Rémunération_Année = sub_montant.find("annee").get_text(strip=True) if sub_montant.find("annee") else None
                        Rémunération_Montant = sub_montant.find("montant").get_text(strip=True).replace("\xa0", "").replace(" ", "") \
                                            if sub_montant.find("montant") else None

                        # Conversion Rémunération_Montant en float
                        try:
                            Rémunération_Montant = float(Rémunération_Montant)
                        except (ValueError, TypeError):
                            continue  # Si la conversion échoue, on saute cette ligne

                        # Stocker la ligne (chaque combinaison montant/année est une ligne distincte)
                        rowi = {
                            "Uuid": Uuid,
                            "Type": section.name,  
                            "ID": ID,
                            "Label": Label,
                            "Commentaire": Commentaire,
                            "Conservee": Conservee,
                            "Description": Description,
                            "Employeur": Employeur,
                            "Date_Début": Date_Début,
                            "Date_Fin": Date_Fin,
                            "Rémunération_Année": Rémunération_Année,
                            "Rémunération_Montant": Rémunération_Montant,
                            "Rémunération_Type": Rémunération_Type
                        }

                        data_items.append(rowi)

dfi = pd.DataFrame(data_items)

In [ ]:
# Filtrer les lignes où Description et Employeur ne sont pas 'neant' ou 'aucun'
# l'élu a rempli un item mais sans information pertinente
dfi = dfi[
    ~(
        (dfi['Description'].str.lower().isin(['neant', 'aucune','0'])) &
        (dfi['Employeur'].str.lower().isin(['neant', 'aucun','0']))
    )
]
dfi

In [ ]:
dfi.info()

In [ ]:
dfi.to_csv("dfi", sep='%', encoding='utf-8', index=False)

In [ ]:
# dfi = pd.read_csv("dfi", sep=';', encoding='utf-8')
# dfg = pd.read_csv("dfg", sep=';', encoding='utf-8')

In [ ]:
# On merge les 2 dataframes navec Uuid en clé primaire commune
df = pd.merge(dfg, dfi, on="Uuid", how="right")
df

In [ ]:
df.to_csv("df", sep='%', encoding='utf-8', index=False)

In [ ]:
# df = pd.read_csv("df", sep='%', encoding='utf-8')

/tmp/ipykernel_15880/1935786925.py:1: DtypeWarning: Columns (21,25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("df", sep='%', encoding='utf-8')


In [6]:
df

,Uuid,DateDepot,Origine,Complete,Civilite,Nom,Prénom,Date_Naissance,Type_Déclaration,Type_Nom_Déclaration,...,Label,Commentaire,Conservee,Description,Employeur,Date_Début,Date_Fin,Rémunération_Année,Rémunération_Montant,Rémunération_Type
0,8e8182a2-ebd7-4b97-84ef-ab01dd6b6c9d,20/09/2021 13:41:36,ADEL,True,Mme,ABADIE,Joelle,23/03/1965,DI,Déclaration d'intérêts,...,NaN,Je perçois à mi-temps 963€ mensuels,NaN,Enseignante,Ministère de l'agriculture,07/2021,NaN,2021,2889.0,Net
1,8e8182a2-ebd7-4b97-84ef-ab01dd6b6c9d,20/09/2021 13:41:36,ADEL,True,Mme,ABADIE,Joelle,23/03/1965,DI,Déclaration d'intérêts,...,NaN,Il s'agit de mon indemnité nette mensuelle (28...,NaN,Vice-Présidente SMECTOM,NaN,07/2020,NaN,2020,1734.0,Net
2,8e8182a2-ebd7-4b97-84ef-ab01dd6b6c9d,20/09/2021 13:41:36,ADEL,True,Mme,ABADIE,Joelle,23/03/1965,DI,Déclaration d'intérêts,...,NaN,Il s'agit de mon indemnité nette mensuelle (28...,NaN,Vice-Présidente SMECTOM,NaN,07/2020,NaN,2021,2601.0,Net
3,8e8182a2-ebd7-4b97-84ef-ab01dd6b6c9d,20/09/2021 13:41:36,ADEL,True,Mme,ABADIE,Joelle,23/03/1965,DI,Déclaration d'intérêts,...,NaN,Il s'agit de mon indemnité nette mensuelle (17...,NaN,Vice-Présidente Département,NaN,07/2021,NaN,2021,5385.0,Net
4,8e8182a2-ebd7-4b97-84ef-ab01dd6b6c9d,20/09/2021 13:41:36,ADEL,True,Mme,ABADIE,Joelle,23/03/1965,DI,Déclaration d'intérêts,...,NaN,Il s'agit de mon indemnité nette mensuelle (46...,NaN,Vice-Présidente CCPL,NaN,07/2020,NaN,2020,2778.0,Net
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312790,19140875-1488-43e7-95a7-63d0b7212a19,25/06/2021 13:29:08,ADEL,True,M.,zuili,nicolas,08/03/1965,DI,Déclaration d'intérêts modificative,...,NaN,Fonction non rémunérée depuis la création de l...,NaN,location d'immeubles,SCI \n [Données non publiées],12/2015,12/2021,2017,0.0,Net
312791,19140875-1488-43e7-95a7-63d0b7212a19,25/06/2021 13:29:08,ADEL,True,M.,zuili,nicolas,08/03/1965,DI,Déclaration d'intérêts modificative,...,NaN,Fonction non rémunérée depuis la création de l...,NaN,location d'immeubles,SCI \n [Données non publiées],12/2015,12/2021,2018,0.0,Net
312792,19140875-1488-43e7-95a7-63d0b7212a19,25/06/2021 13:29:08,ADEL,True,M.,zuili,nicolas,08/03/1965,DI,Déclaration d'intérêts modificative,...,NaN,Fonction non rémunérée depuis la création de l...,NaN,location d'immeubles,SCI \n [Données non publiées],12/2015,12/2021,2019,0.0,Net
312793,19140875-1488-43e7-95a7-63d0b7212a19,25/06/2021 13:29:08,ADEL,True,M.,zuili,nicolas,08/03/1965,DI,Déclaration d'intérêts modificative,...,NaN,Fonction non rémunérée depuis la création de l...,NaN,location d'immeubles,SCI \n [Données non publiées],12/2015,12/2021,2020,0.0,Net


Aggregations Exemples

In [7]:
rem_annee = df.groupby(['Nom','Prénom', 'Rémunération_Année']).agg({'Rémunération_Montant': 'sum'}).reset_index()
rem_annee

,Nom,Prénom,Rémunération_Année,Rémunération_Montant
0,ABADIE,Joelle,2020,9391.0
1,ABADIE,Joelle,2021,21315.0
2,ABBAS,Gérard,2015,312.0
3,ABBAS,Gérard,2016,546.0
4,ABBAS,Gérard,2017,312.0
...,...,...,...,...
33243,zuili,nicolas,2017,0.0
33244,zuili,nicolas,2018,0.0
33245,zuili,nicolas,2019,0.0
33246,zuili,nicolas,2020,0.0


In [8]:
rem_pivot = rem_annee.pivot(index=['Nom', 'Prénom'], columns='Rémunération_Année', values='Rémunération_Montant')
rem_pivot = rem_pivot.reset_index()
rem_pivot

Rémunération_Année,Nom,Prénom,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,ABADIE,Joelle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9391.0,21315.0,NaN,NaN,NaN,NaN
1,ABBAS,Gérard,NaN,NaN,NaN,NaN,NaN,312.0,546.0,312.0,0.0,858.0,468.0,0.0,NaN,NaN,NaN,NaN
2,ABELIN-DRAPRON,Véronique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,ABELLI,Thierry,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,ABINAL,Jean-Philippe,NaN,NaN,NaN,NaN,NaN,NaN,87866.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5046,yvon,richard,NaN,NaN,NaN,NaN,136534.0,130083.0,156084.0,134012.0,125130.0,130300.0,0.0,NaN,NaN,NaN,NaN,NaN
5047,zanardo,jacky,NaN,121270.0,122900.0,129160.0,124972.0,124214.0,127842.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5048,zedet,christian,NaN,NaN,NaN,NaN,NaN,63780.0,66447.0,67192.0,67200.0,66900.0,49900.0,NaN,NaN,NaN,NaN,NaN
5049,zerkani,karima,NaN,NaN,NaN,NaN,0.0,72495.0,98352.0,129165.0,222723.0,166104.0,105639.0,NaN,NaN,NaN,NaN,NaN


In [20]:
mandat = df[df['Type'] == 'mandatelectifdto']
rem_mandat = mandat.groupby(['Nom','Prénom', 'Type_Mandat', 'Date_début_Mandat']).agg({'Rémunération_Montant': 'sum'}).reset_index()
rem_mandat

,Nom,Prénom,Type_Mandat,Date_début_Mandat,Rémunération_Montant
0,ABADIE,Joelle,Elu départemental,23/07/2021,27817.0
1,ABBAS,Gérard,Elu départemental,01/07/2021,0.0
2,ABELIN-DRAPRON,Véronique,Elu départemental,27/06/2021,0.0
3,ABINAL,Jean-Philippe,Elu départemental,06/08/2021,13405.0
4,ABINAL,Jean-Philippe,Elu départemental,29/07/2021,13405.0
...,...,...,...,...,...
4718,vogel,jean pierre,Sénateur,27/09/2020,170244.0
4719,walker,lionel,Membre d’EPCI,10/07/2020,2590.0
4720,zedet,christian,Membre d’EPCI,01/08/2020,9900.0
4721,zerkani,karima,Maire ou adjoint municipal,23/06/2020,194716.0


Partie source Christophe Goudet


TESTS

In [ ]:
test = [
    x
    for x in declarations
    if x.find("uuid").text or "" == "ed7219fa-b580-46ea-b430-ceeb22c1f223"
]
test[0]

In [ ]:
sorted(x.name for x in test[0].find_all(recursive=False))

PARSER

In [ ]:
def _parse_mandat_revenues(declarations: BeautifulSoup) -> list[dict]:
    section = declarations.find("mandatelectifdto")
    if not section:
        return []

    remunerations = section.find("items")
    if not remunerations:
        return []

    remuneration = remunerations.find("remuneration")
    general_infos = {
        "description_mandat": section.find("descriptionmandat").text,
    }
    montants = remuneration.find("montant", recursive=False)
    return [
        general_infos
        | {"montant": float(item.find("montant").text.replace("\xa0", ""))}
        for item in montants.find_all("montant", recursive=False)
    ]

In [ ]:
_parse_mandat_revenues(test[0])

Voici le [lien](https://www.hatvp.fr/wordpress/wp-content/uploads/2017/07/opendata-structure.xlsx) vers le fichier Excel contenant la structure des données XML

In [ ]:
# Structure du fichier XML

structure_df = pd.read_excel("https://www.hatvp.fr/wordpress/wp-content/uploads/2017/07/opendata-structure.xlsx")
structure_df

Le Xpath contient le chemin de l'élément dans l'arbre XML. 
On peut utiliser ce chemin pour requêter tous les éléments correspondant au champ


In [ ]:
# Exemple de requêtage grâce à au Xpath
# Rappel : root est l'élément racine de l'arbre XML, il est défini dans la deuxième cellule de code
# Pour tester : récupérer le Xpath plus haut et remplacer la valeur de la variable xpath 

xpath = "/declaration/general/declarant/nom"

for element in root.findall("." + xpath):
    print(element.text)

Etude de Fonctions et mandats électifs

In [ ]:
REMpath = "/declaration/mandatElectifDto/items/items/remuneration/brutNet"

for element in root.findall("." + REMpath):
    print(element.text)

In [ ]:
dict_xpath = {}

for index, row in structure_df.iloc[51:62].iterrows():
    dict_xpath[row["Champs"]] = row["Xpath"]

print(dict_xpath)


In [ ]:
dict_personne = {}

for index, row in structure_df.iloc[128:129].iterrows():
    dict_personne[row["Champs"]] = row["Xpath"]
    for index, row in structure_df.iloc[51:62].iterrows():
        dict_xpath[row["Champs"]] = row["Xpath"]

print(dict_personne)